# Purpose
Series of steps to test for leakage in an existing project.

The example below is applied to the [Dirty Duck tutorial](https://github.com/dssg/dirtyduck), using City of Chicago inspections prioritization data. The variables in ALL CAPS are things that need to be customized for each project, so make a copy of this notebook and personalize it for your project.

# Process



In [2]:
import sys
import os
import math
import warnings

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine

from sklearn import linear_model
from sklearn import metrics 
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from IPython.core.interactiveshell import InteractiveShell

from dd_credentials import * # credentials file containing database login variables (dbname, user, host, password, port)

warnings.filterwarnings(action='once')
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def execute_sql(statement, dbname, user, host, password, port, isolation = False, results = True):
    """
    Use psycopg2 to execute PostgreSQL queries
    
    Input:
        statement (str): SQL statement to run in database
        dbname, user, host, password (str): database credentials
        isolation (bool): indicator for whether to change isolation level to autocommit; True for queries that cannot be run 
            from within a transation (see https://wiki.postgresql.org/wiki/Psycopg2_Tutorial), e.g. queries other than SELECT
        results (bool): indicator for whether the query is expected to output results;
            for example, True for SELECT statements and False for CREATE TABLE statements
    
    Output:
        relation (dataframe): query results or empty dataframe if results = False
    """
    conn = psycopg2.connect("dbname={} user={} host={} password={} port={}".format(dbname, user, host, password, port))
    cur = conn.cursor()
    if isolation:
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur.execute(statement)
    relation = pd.DataFrame()
    if results:
        results = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        relation = pd.DataFrame(results, columns=colnames)
    cur.close()
    conn.close()
    return relation


In [4]:
def randomize(df, do_not_randomize = None):
    """
    Randomize column values of a file. Each column is randomized independently.
    
    Inputs:
        df (dataframe): dataframe to randomize
        do_not_randomize (list): optional list of strings indicating names of 
            columns that should not be randomized
    Outputs:
        df (dataframe): dataframe of randomized data
    """     
    df_random = df.copy()
    if do_not_randomize:
        cols = [c for c in df.columns if c not in do_not_randomize]
    else:
        cols = df.columns
        
    for col in cols:
        #print('\t\tRandomizing column ' + col)
        df_random[col] = np.random.permutation(df_random[col])

    return df_random

In [5]:
# source: https://overlaid.net/2016/02/08/replace-words-in-files-or-strings-using-python/
def do_replacement(base_text, word_map):
    """
    Helper function for replace_words_in_file. Make replacements in base_text, as 
    indicated in  word_map.
    """
    for key, val in word_map.items():
        base_text = base_text.replace(key, val)
    return base_text

def replace_words_in_file(read_from, write_to, word_map):
    """
    Create copy of a file with certain words replaced
    
    Inputs
        read_from: name of file to read from
        write_to: name of new file to be created
        word_map: dictionary of mappings between words and their replacements
            (e.g. {'old_word': 'new_word'})
    Outputs
        None. Will create a new file with the name given in write_to
        
    """
    # Open your desired file as 't' and read the lines into string 'tempstr'
    t = open(read_from, 'r')
    tempstr = t.read()
    t.close()

    # Using the "replace_words" function, we'll pass in our tempstr to be used as the base, 
    # and our device_values to be used as replacement.  
    output = do_replacement(tempstr, word_map)

    # Write out the new config file
    fout = open(write_to, 'w')
    fout.write(output)
    fout.close()

In [6]:
def get_new_filename(filename, suffix):
    """
    Output a new filename (str) given a filename and suffix to append. Assumes file extension at the end is separated by a period.
    """
    k = filename.rfind(".")
    new_filename = filename[:k] + suffix + '.' + filename[k+1:]
    return new_filename

## I. Create a randomized version of the raw schema

Pull the information schema from selected database.

In [5]:
# name of the database schema containing raw data to be randomized
RAW_SCHEMA = 'raw' 

In [5]:
statement = "SELECT * FROM information_schema.tables;"
tables = execute_sql(statement, dbname, user, host, password, port)

In [6]:
tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,food,postgis,geography_columns,VIEW,None,None,None,None,None,NO,NO,None
1,food,postgis,geometry_columns,VIEW,None,None,None,None,None,YES,NO,None
2,food,postgis,spatial_ref_sys,BASE TABLE,None,None,None,None,None,YES,NO,None
3,food,postgis,raster_columns,VIEW,None,None,None,None,None,NO,NO,None
4,food,postgis,raster_overviews,VIEW,None,None,None,None,None,NO,NO,None
5,food,cleaned,inspections,BASE TABLE,None,None,None,None,None,YES,NO,None
6,food,pg_catalog,pg_statistic,BASE TABLE,None,None,None,None,None,YES,NO,None
7,food,pg_catalog,pg_type,BASE TABLE,None,None,None,None,None,YES,NO,None
8,food,cleaned,violations,BASE TABLE,None,None,None,None,None,YES,NO,None
9,food,pg_catalog,pg_policy,BASE TABLE,None,None,None,None,None,YES,NO,None


Select tables from the raw schema; they will be randomized. In this tutorial, there is only one (raw.inspections).

In [7]:
raw = tables[tables.table_schema == RAW_SCHEMA]
raw.head()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
12,food,raw,inspections,BASE TABLE,None,None,None,None,None,YES,NO,None


First created the _randomized schema if it doesn't yet exist

In [14]:
statement = "CREATE SCHEMA IF NOT EXISTS {}_randomized;".format(RAW_SCHEMA)
output = execute_sql(statement, dbname, user, host, password, port, isolation=True, results = False)

Randomize every table in raw and write output to _randomized schema

In [15]:
for table_name in raw.table_name:
    print("Working on table {}".format(table_name))
    
    # Pull the table from original schema
    print("\tPulling table")
    statement = "SELECT * FROM {}.{};".format(RAW_SCHEMA, table_name)
    table = execute_sql(statement, dbname, user, host, password, port)
    
    # Randomize the table
    print("\tRandomizing")
    randomized_table = randomize(table)
    
    # Make a new table in _randomized schema
    print("\tUploading randomized version")
    statement = "CREATE TABLE IF NOT EXISTS {}_randomized.{} (LIKE {}.{});".format(RAW_SCHEMA, table_name, RAW_SCHEMA, table_name)
    output = execute_sql(statement, dbname, user, host, password, port, isolation = True, results = False)
    
    # Write results into new table
    statement = "SELECT COUNT(*) FROM {}_randomized.{};".format(RAW_SCHEMA, table_name)
    output = execute_sql(statement, dbname, user, host, password, port)
    if output.iloc[0,0]>0: #  do nothing if new table already contains data
        print("\t*****SKIPPING TABLE {} -- it already has data".format(table_name))
    else:
        engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))
        randomized_table.to_sql(table_name, engine, schema = '{}_randomized'.format(RAW_SCHEMA), index = False, if_exists='append')
    

Working on table inspections
	Pulling table
	Randomizing
	Uploading randomized version


## II. Edit + execute preprocessing and config files so that they point to the newly randomized raw schema

New versions of the preprocessing and config files with an "_edited" suffix will be produced.

Make list of preprocessing files that get RAW_SCHEMA ready for triage experiments. The list should be in correct execution order, and file names should have some extension (e.g. .sql).

NOTE: The code below currently only works for .sql files. Need to add .py flexibility if necessary.

In [16]:
PREPROC_FILES = ['/home/ubuntu/dirtyduck/sql/create_cleaned_inspections_table.sql',
                 '/home/ubuntu/dirtyduck/sql/create_violations_table.sql',
                 '/home/ubuntu/dirtyduck/sql/create_semantic_tables.sql',
                 '/home/ubuntu/dirtyduck/sql/create_inspections_schema.sql'
                ]

Also define the experiment config file name

In [ ]:
EXP_CONFIG = "/home/ubuntu/dirtyduck/triage/experiment_config/inspections_test.yaml"

Make a dictionary of text replacements to apply across every preprocessing & config file. This should include:
- Names of schemas that are produced during preprocessing. Add a _randomized suffix to each of them.
- The "purpose" model group key from the experiment config file. In order to avoid overwriting existing model groups, the purpose must be changed to indicate that this experiment is for leakage detection.



In [23]:
WORD_MAP = {RAW_SCHEMA+'.': RAW_SCHEMA+'_randomized.', 
    'cleaned.': 'cleaned_randomized.', 
    'semantic.': 'semantic_randomized.',
    'inspections.': 'inspections_randomized.',
    "purpose: 'test'": "purpose: 'leakage-detection'"}

Apply find+replace to every preprocessing file.  

**NOTE**: Be sure to open the outputted files (with "_edited" suffix) to check that no unexpected replacements were made and that all necessary replacements are made. In this case, we have to manually edit some schema creation statements.

In [30]:
preproc_files_edited = []
for filename in PREPROC_FILES:
    new_filename = get_new_filename(filename, "_edited")
    replace_words_in_file(filename, new_filename, WORD_MAP)
    preproc_files_edited.append(new_filename)

Execute the edited preprocessing files

In [31]:
preproc_files_edited

['/home/ubuntu/dirtyduck/sql/create_cleaned_inspections_table_edited.sql',
 '/home/ubuntu/dirtyduck/sql/create_violations_table_edited.sql',
 '/home/ubuntu/dirtyduck/sql/create_semantic_tables_edited.sql',
 '/home/ubuntu/dirtyduck/sql/create_inspections_schema_edited.sql']

In [22]:
for filename in preproc_files_edited:
    print("Executing file ", filename)
    conn = psycopg2.connect("dbname={} user={} host={} password={} port={}".format(dbname, user, host, password, port))
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    with conn.cursor() as cursor:
        with open(filename, 'r') as f:
            cursor.execute(f.read())

Executing file  /home/ubuntu/dirtyduck/sql/create_cleaned_inspections_table_edited.sql
Executing file  /home/ubuntu/dirtyduck/sql/create_violations_table_edited.sql
Executing file  /home/ubuntu/dirtyduck/sql/create_semantic_tables_edited.sql
Executing file  /home/ubuntu/dirtyduck/sql/create_inspections_schema_edited.sql


Now edit the experiment config file in a similar fashion. Apply find+replace to config file.  

**NOTE**: Again, be sure to open the outputted file (with "_edited" suffix) to check that no unexpected replacements were made and that all necessary replacements are made.

In [32]:
new_exp_config = get_new_filename(EXP_CONFIG, '_edited')
replace_words_in_file(EXP_CONFIG, new_exp_config, WORD_MAP)

## III. Run an experiment with this new setup

Now that we've set up randomized versions of the original schemas in the database and created a new config file for a randomized experiment, it's time to run the actual triage experiment.

**NOTE**: This must be done from the command line!

- First validate the new config file. For this tutorial: ./tutorial.sh triage --config_file inspections_test_edited.yaml validate
- Then execute the experiment. For this tutorial: ./tutorial.sh triage --config_file inspections_test_edited.yaml run

## IV. Align the results of randomized experiment against original experiment

First, we need to map the model groups from our new, randomized experiment to the model groups from the original experiment

Confirm that we now see a set of model groups where purpose = "leakage-detection"

In [121]:
statement = "SELECT * FROM results.model_groups;"
model_groups = execute_sql(statement, dbname, user, host, password, port)
model_groups

,model_group_id,model_type,model_parameters,feature_list,model_config
0,1,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
1,2,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 'sqrt'}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
2,3,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': None}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
3,4,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
4,5,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': 'sqrt'}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
5,6,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': None}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
6,7,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."
7,8,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 'sqrt'}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."
8,9,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': None}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."
9,10,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."


Recall that a model group is defined by the classifier (model_type), its parameters (model_parameters), the features (feature_list), and the model_config. Since we varied the "purpose" key within model_config for these randomized experiments, all other properties of a model group should be used as index keys. This includes other model group keys in the model_config column.

In [122]:
# pull out all model group keys from model_config column and make new columns from them
list_of_lists = list(model_groups.model_config.apply(lambda x: list(x.keys())))
keys = set([key for x in list_of_lists for key in x])

for key in keys:
    model_groups[key] = model_groups.model_config.apply(lambda x: x.get(key, None))

In [123]:
# convert model_parameters and feature_list columns from dictionaries to strings so they can be used as index keys
model_groups['model_parameters'] = model_groups.model_parameters.apply(lambda x: ', '.join('{}{}'.format(key, val) for key, val in x.items()))
model_groups['feature_list'] = model_groups.feature_list.apply(lambda x: ', '.join(x))

In [124]:
index_cols = [col for col in model_groups.columns if col not in ['purpose', 'model_group_id', 'model_config']]
index_cols

['model_type',
 'model_parameters',
 'feature_list',
 'label_definition',
 'experiment_type']

In [125]:
model_groups = model_groups.set_index(index_cols)

Join the original and randomized model groups to create a wide (rather than long) version of model_group table. Each row of model_groups_wide corresponds to a particular model group configuration and indicates the model group ids of the original vs randomized versions.

In [147]:
# define original "purpose" model group key
ORIG_PURPOSE = 'test'

In [126]:
orig = model_groups[model_groups.purpose == ORIG_PURPOSE]
rand = model_groups[model_groups.purpose == 'leakage-detection']
model_groups_wide = pd.merge(orig[['model_group_id']], rand[['model_group_id']], how='right', left_index=True, right_index=True, suffixes = ['_orig', '_rand'])

In [127]:
model_groups_wide

model_group_id_orig  \
model_type                          model_parameters                feature_list                                       label_definition experiment_type                                   
sklearn.tree.DecisionTreeClassifier max_depth1, max_features1       inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    1   
                                    max_depth1, max_featuressqrt    inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    2   
                                    max_depth1, max_featuresNone    inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    3   
                                    max_depthNone, max_features1    inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    4   
                                    max_depthNone, max_featuressqrt inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    5   
                                    max_depthNone, max_featuresNone inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    6   

                                                                                                                                                                    model_group_id_rand  
model_type                          model_parameters                feature_list                                       label_definition experiment_type                                  
sklearn.tree.DecisionTreeClassifier max_depth1, max_features1       inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    7  
                                    max_depth1, max_featuressqrt    inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    8  
                                    max_depth1, max_featuresNone    inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                    9  
                                    max_depthNone, max_features1    inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                   10  
                                    max_depthNone, max_featuressqrt inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                   11  
                                    max_depthNone, max_featuresNone inspections_entity_id_3month_type_canvass_sum, ... failed           inspections prioritization                   12

Pick an experiment on which to test for leakage. Let's go with the first row: model_group_id_orig = 1, model_group_id_rand = 7

In [128]:
# create variables for the model_group id's you selected
ID_ORIG = 1
ID_RAND = 7

Select the models relevant to each model group

In [129]:
statement = "SELECT * FROM results.models where model_group_id = {};".format(ID_ORIG)
models_orig = execute_sql(statement, dbname, user, host, password, port)
models_orig

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,training_label_timespan
0,1,1,a1a3b6502fb1e63ad80d09a8784b9c0d,2018-03-08 20:41:43.170099,2018-03-08 20:41:43.165540,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,7bc94b4256e8a40715be5551934d31ff,2015-12-13,False,c939d74a584837f89f8213cb2624809a,30 days
1,7,1,280fc4f7ade3987a996a6692f72b572b,2018-03-08 20:42:34.884985,2018-03-08 20:42:34.881412,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,7bc94b4256e8a40715be5551934d31ff,2016-12-13,False,b1acc63526a1347ded8daacf449ec51f,30 days


In [130]:
statement = "SELECT * FROM results.models where model_group_id = {};".format(ID_RAND)
models_rand = execute_sql(statement, dbname, user, host, password, port)
models_rand

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,training_label_timespan
0,13,7,440ed05c425d947f13656439e6d862ed,2018-03-08 22:59:16.086865,2018-03-08 22:59:16.082227,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,e2cebd41a9610c55b704b115741695a0,2015-12-13,False,e35a116afe12b47979e33ee74060a208,30 days
1,19,7,376c959f493b2285a858080553f22edc,2018-03-08 23:04:02.859653,2018-03-08 23:04:02.856651,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,e2cebd41a9610c55b704b115741695a0,2016-12-13,False,4da99c0c6b757bc3ff9fa1e6781c2468,30 days


Grab the predictions made by each of these models.

In [140]:
statement = "SELECT * FROM results.predictions where model_id in {};".format(tuple(models_orig.model_id))
predictions_orig = execute_sql(statement, dbname, user, host, password, port)
predictions_orig

,model_id,entity_id,as_of_date,score,label_value,rank_abs,rank_pct,matrix_uuid,test_label_timespan
0,1,1,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
1,1,2,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
2,1,5,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
3,1,6,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
4,1,7,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
5,1,8,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
6,1,9,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
7,1,10,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
8,1,13,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days
9,1,14,2015-12-13,0.23261973074376519,NaN,None,None,cc5efa865f14af126cee374e3b7dc9f2,30 days


In [141]:
statement = "SELECT * FROM results.predictions where model_id in {};".format(tuple(models_rand.model_id))
predictions_rand = execute_sql(statement, dbname, user, host, password, port)
predictions_rand

,model_id,entity_id,as_of_date,score,label_value,rank_abs,rank_pct,matrix_uuid,test_label_timespan
0,13,1,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
1,13,2,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
2,13,4,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
3,13,5,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
4,13,6,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
5,13,8,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
6,13,9,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
7,13,10,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
8,13,12,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days
9,13,14,2015-12-13,0.23220973782771537,NaN,None,None,febde1976e96e5234ec15058565d73c4,30 days


Grab the evaluation metrics for each of these models.

In [143]:
statement = "SELECT * FROM results.evaluations where model_id in {};".format(tuple(models_orig.model_id))
evaluations_orig = execute_sql(statement, dbname, user, host, password, port)
evaluations_orig

,model_id,evaluation_start_time,evaluation_end_time,as_of_date_frequency,metric,parameter,value,num_labeled_examples,num_labeled_above_threshold,num_positive_labels,sort_seed
0,1,2015-12-13,2015-12-13,30 days,precision@,5.0_pct,0.2553191489361702,792,47,181,5
1,1,2015-12-13,2015-12-13,30 days,recall@,5.0_pct,0.06629834254143646,792,47,181,5
2,1,2015-12-13,2015-12-13,30 days,precision@,10.0_pct,0.23655913978494625,792,93,181,5
3,1,2015-12-13,2015-12-13,30 days,recall@,10.0_pct,0.12154696132596685,792,93,181,5
4,1,2015-12-13,2015-12-13,30 days,precision@,5_abs,0.0,792,0,181,5
5,1,2015-12-13,2015-12-13,30 days,recall@,5_abs,0.0,792,0,181,5
6,1,2015-12-13,2015-12-13,30 days,precision@,10_abs,0.0,792,0,181,5
7,1,2015-12-13,2015-12-13,30 days,recall@,10_abs,0.0,792,0,181,5
8,1,2015-12-13,2015-12-13,30 days,precision@,25_abs,0.0,792,0,181,5
9,1,2015-12-13,2015-12-13,30 days,recall@,25_abs,0.0,792,0,181,5


In [144]:
statement = "SELECT * FROM results.evaluations where model_id in {};".format(tuple(models_rand.model_id))
evaluations_rand = execute_sql(statement, dbname, user, host, password, port)
evaluations_rand

,model_id,evaluation_start_time,evaluation_end_time,as_of_date_frequency,metric,parameter,value,num_labeled_examples,num_labeled_above_threshold,num_positive_labels,sort_seed
0,13,2015-12-13,2015-12-13,30 days,precision@,5.0_pct,0.0,0,0,0,5
1,13,2015-12-13,2015-12-13,30 days,recall@,5.0_pct,0.0,0,0,0,5
2,13,2015-12-13,2015-12-13,30 days,precision@,10.0_pct,0.0,0,0,0,5
3,13,2015-12-13,2015-12-13,30 days,recall@,10.0_pct,0.0,0,0,0,5
4,13,2015-12-13,2015-12-13,30 days,precision@,5_abs,0.0,0,0,0,5
5,13,2015-12-13,2015-12-13,30 days,recall@,5_abs,0.0,0,0,0,5
6,13,2015-12-13,2015-12-13,30 days,precision@,10_abs,0.0,0,0,0,5
7,13,2015-12-13,2015-12-13,30 days,recall@,10_abs,0.0,0,0,0,5
8,13,2015-12-13,2015-12-13,30 days,precision@,25_abs,0.0,0,0,0,5
9,13,2015-12-13,2015-12-13,30 days,recall@,25_abs,0.0,0,0,0,5


## V. Perform statistical tests on the randomized model to see if it performs better than random (and thus suggests data leakage)

Tutorial next steps:
- Bootstrap the predicted scores to get a large sample of certain evaluation metrics
- Perform statistical tests for leakage on those evaluation metrics

Note: Dirty duck tutorial example may not be the best demonstration of the statistical tests; there is almost no variation in the predicted scores. May need to edit the experiment setup to fit a different model; will talk to Adolfo.